# Demo Notebook - July 10th

This notebook demonstrates the building blocks that lead to Query 0 (Genome Feature to Reaction Mapping) and explores comparative genomics across 50 E. coli strains.

## Part 1: Building Blocks for Query 0

Let's explore each component that makes the genome-to-reaction mapping possible.

In [1]:
# Setup - Import required libraries and initialize Spark
from spark.utils import get_spark_session
import time
import pandas as pd
from IPython.display import display

spark = get_spark_session()
namespace = 'ontology_data'

# Helper function to time queries
def time_query(query_name, query_func):
    """Execute a query and print execution time"""
    print(f"\n{'='*60}")
    print(f"Executing: {query_name}")
    print(f"{'='*60}")
    start_time = time.time()
    result = query_func()
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\nQuery execution time: {execution_time:.2f} seconds")
    return result

25/07/10 05:11:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 05:11:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/07/10 05:11:30 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark-job-logs/jplfaria/app-20250710051128-0023.inprogress. This is unsupported
25/07/10 05:11:30 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


### 1. Exploring SEED Reactions in the Ontology

SEED reactions are stored in the statements table with their labels. Let's see what they look like:

In [9]:
def explore_seed_reactions():
    query = f"""
    SELECT 
        subject as reaction_id,
        predicate,
        value as reaction_name
    FROM {namespace}.statements
    WHERE subject LIKE 'seed.reaction:%'
    AND predicate = 'rdfs:label'
    LIMIT 10
    """
    
    df = spark.sql(query).toPandas()
    print(f"Sample SEED reactions from the ontology:")
    display(df)
    
    # Count total reactions
    count_query = f"""
    SELECT COUNT(DISTINCT subject) as total_reactions
    FROM {namespace}.statements
    WHERE subject LIKE 'seed.reaction:%'
    AND predicate = 'rdfs:label'
    """
    count = spark.sql(count_query).collect()[0]['total_reactions']
    print(f"\nTotal SEED reactions in ontology: {count:,}")

# Execute without capturing return value to avoid duplicate display
time_query("Explore SEED Reactions", explore_seed_reactions)


Executing: Explore SEED Reactions


Sample SEED reactions from the ontology:


,reaction_id,predicate,reaction_name
0,seed.reaction:rxn00001,rdfs:label,diphosphate phosphohydrolase
1,seed.reaction:rxn00002,rdfs:label,urea-1-carboxylate amidohydrolase
2,seed.reaction:rxn00003,rdfs:label,pyruvate:pyruvate acetaldehydetransferase (dec...
3,seed.reaction:rxn00004,rdfs:label,4-hydroxy-4-methyl-2-oxoglutarate pyruvate-lya...
4,seed.reaction:rxn00006,rdfs:label,hydrogen-peroxide:hydrogen-peroxide oxidoreduc...
5,seed.reaction:rxn00007,rdfs:label,"alpha,alpha-trehalose glucohydrolase"
6,seed.reaction:rxn00008,rdfs:label,Mn(II):hydrogen-peroxide oxidoreductase
7,seed.reaction:rxn00009,rdfs:label,GTP:GTP guanylyltransferase
8,seed.reaction:rxn00010,rdfs:label,glyoxylate carboxy-lyase (dimerizing; tartrona...
9,seed.reaction:rxn00011,rdfs:label,pyruvate:thiamin diphosphate acetaldehydetrans...



Total SEED reactions in ontology: 44,901

Query execution time: 2.65 seconds


### 2. Exploring SEED Roles in the Ontology

SEED roles represent enzyme functions. Let's examine them:

In [3]:
def explore_seed_roles():
    query = f"""
    SELECT 
        subject as role_id,
        predicate,
        value as role_name
    FROM {namespace}.statements
    WHERE subject LIKE 'seed.role:%'
    AND predicate = 'rdfs:label'
    LIMIT 10
    """
    
    df = spark.sql(query).toPandas()
    print(f"Sample SEED roles (enzyme functions) from the ontology:")
    display(df)
    
    # Count total roles
    count_query = f"""
    SELECT COUNT(DISTINCT subject) as total_roles
    FROM {namespace}.statements
    WHERE subject LIKE 'seed.role:%'
    AND predicate = 'rdfs:label'
    """
    count = spark.sql(count_query).collect()[0]['total_roles']
    print(f"\nTotal SEED roles in ontology: {count:,}")

time_query("Explore SEED Roles", explore_seed_roles)


Executing: Explore SEED Roles


Sample SEED roles (enzyme functions) from the ontology:


,role_id,predicate,role_name
0,seed.role:0000000000001,rdfs:label,(+)-caryolan-1-ol synthase (EC 4.2.1.138)
1,seed.role:0000000000002,rdfs:label,"(2E,6E)-farnesyl diphosphate synthase (EC 2.5...."
2,seed.role:0000000000003,rdfs:label,"(2E,6Z)-farnesyl diphosphate synthase (EC 2.5...."
3,seed.role:0000000000004,rdfs:label,(2R)-sulfolactate sulfo-lyase subunit alpha (E...
4,seed.role:0000000000005,rdfs:label,(2R)-sulfolactate sulfo-lyase subunit beta (EC...
5,seed.role:0000000000006,rdfs:label,(3R)-hydroxyacyl-ACP dehydratase subunit HadA
6,seed.role:0000000000007,rdfs:label,(3R)-hydroxyacyl-ACP dehydratase subunit HadB
7,seed.role:0000000000008,rdfs:label,(3R)-hydroxyacyl-ACP dehydratase subunit HadC
8,seed.role:0000000000011,rdfs:label,(Carboxyethyl)arginine beta-lactam-synthase (E...
9,seed.role:0000000000013,rdfs:label,"(R)-2-hydroxyacid dehydrogenase, similar to L-..."


[Stage 19:======================================>                   (2 + 1) / 3]


Total SEED roles in ontology: 15,383

Query execution time: 7.46 seconds


### 3. Understanding Term Associations: How Roles Map to Reactions

The term_association table connects SEED roles to reactions they catalyze:

In [10]:
def explore_term_associations():
    query = f"""
    WITH role_reaction_mappings AS (
        -- Get role to reaction mappings
        SELECT 
            ta.subject as role_string,
            ta.predicate,
            ta.object as reaction_id
        FROM {namespace}.term_association ta
        WHERE ta.object LIKE 'seed.reaction:%'
        AND ta.predicate = 'RO:0002327'
        LIMIT 5
    )
    SELECT 
        m.role_string,
        m.reaction_id,
        rxn.value as reaction_name,
        m.predicate
    FROM role_reaction_mappings m
    LEFT JOIN {namespace}.statements rxn 
        ON m.reaction_id = rxn.subject AND rxn.predicate = 'rdfs:label'
    ORDER BY m.role_string
    """
    
    df = spark.sql(query).toPandas()
    print(f"Sample role-to-reaction mappings:")
    display(df)
    
    # Show predicate meaning
    print("\nNote: predicate 'RO:0002327' means 'enables' - the role enables/catalyzes the reaction")
    print("\nNote: Role strings in term_association are the actual RAST annotations, not seed.role IDs")

time_query("Explore Term Associations", explore_term_associations)


Executing: Explore Term Associations


Sample role-to-reaction mappings:


,role_string,reaction_id,reaction_name,predicate
0,2-polyprenyl-6-hydroxyphenyl methylase (ec 2.1...,seed.reaction:rxn03395,S-adenosyl-L-methionine:3-(all-trans-octapreny...,RO:0002327
1,D-allose kinase (EC 2.7.1.55),seed.reaction:rxn02552,ATP:D-allose 6-phosphotransferase,RO:0002327
2,Glutamate synthase [NADPH] small chain (EC 1.4...,seed.reaction:rxn00085,L-Glutamate:NADP+ oxidoreductase (transaminating),RO:0002327
3,Succinyl-CoA:3-ketoacid-coenzyme A transferase...,seed.reaction:rxn00290,succinyl-CoA:acetoacetate CoA-transferase,RO:0002327
4,Succinyl-CoA:3-ketoacid-coenzyme A transferase...,seed.reaction:rxn02143,Succinyl-CoA:3-oxoadipate CoA-transferase,RO:0002327



Note: predicate 'RO:0002327' means 'enables' - the role enables/catalyzes the reaction

Note: Role strings in term_association are the actual RAST annotations, not seed.role IDs

Query execution time: 2.64 seconds


### 4. Exploring Feature Annotations: RAST Roles in Genomes

The feature_annotation table contains RAST annotations that match SEED role names:

In [11]:
def explore_feature_annotations():
    query = f"""
    SELECT 
        genome_id,
        feature_id,
        rast,
        bakta_gene,
        bakta_product
    FROM {namespace}.feature_annotation
    WHERE genome_id = '562.61239'
    AND rast IS NOT NULL
    LIMIT 5
    """
    
    df = spark.sql(query).toPandas()
    print(f"Sample RAST annotations from E. coli genome 562.61239:")
    display(df)
    
    # Count features with RAST annotations
    count_query = f"""
    SELECT 
        COUNT(*) as features_with_rast,
        COUNT(DISTINCT rast) as unique_rast_roles
    FROM {namespace}.feature_annotation
    WHERE genome_id = '562.61239'
    AND rast IS NOT NULL
    """
    counts = spark.sql(count_query).collect()[0]
    print(f"\nGenome 562.61239 has:")
    print(f"  - {counts['features_with_rast']:,} features with RAST annotations")
    print(f"  - {counts['unique_rast_roles']:,} unique RAST roles")

time_query("Explore Feature Annotations", explore_feature_annotations)


Executing: Explore Feature Annotations
Sample RAST annotations from E. coli genome 562.61239:


,genome_id,feature_id,rast,bakta_gene,bakta_product
0,562.61239,562.61239_1,Alpha-ketoglutarate permease,None,Alpha-ketoglutarate permease
1,562.61239,562.61239_2,Alpha-ketoglutarate permease,None,hypothetical protein
2,562.61239,562.61239_3,Putative outer membrane lipoprotein,yfiM,YfiM family lipoprotein
3,562.61239,562.61239_4,CDP-diacylglycerol--serine O-phosphatidyltrans...,pssA,CDP-diacylglycerol--serine O-phosphatidyltrans...
4,562.61239,562.61239_5,Protein lysine acetyltransferase Pat (EC 2.3.1.-),pat,protein lysine acetyltransferase



Genome 562.61239 has:
  - 4,410 features with RAST annotations
  - 3,832 unique RAST roles

Query execution time: 2.05 seconds


### 5. Connecting Features to Roles: The Key Link

Let's verify that RAST annotations in feature_annotation match SEED role subjects in term_association:

In [6]:
def verify_rast_to_role_connection():
    query = f"""
    WITH genome_rast_roles AS (
        -- Get unique RAST roles from our genome
        SELECT DISTINCT rast
        FROM {namespace}.feature_annotation
        WHERE genome_id = '562.61239'
        AND rast IS NOT NULL
    ),
    matching_term_associations AS (
        -- Find which RAST roles exist in term_association
        SELECT 
            gr.rast,
            COUNT(DISTINCT ta.object) as reaction_count
        FROM genome_rast_roles gr
        INNER JOIN {namespace}.term_association ta
            ON gr.rast = ta.subject
        WHERE ta.object LIKE 'seed.reaction:%'
        GROUP BY gr.rast
    )
    SELECT 
        rast as role_string,
        reaction_count
    FROM matching_term_associations
    ORDER BY reaction_count DESC
    LIMIT 10
    """
    
    df = spark.sql(query).toPandas()
    print(f"RAST roles that successfully map to reactions:")
    display(df)
    
    # Summary statistics
    stats_query = f"""
    WITH genome_rast AS (
        SELECT DISTINCT rast
        FROM {namespace}.feature_annotation
        WHERE genome_id = '562.61239'
        AND rast IS NOT NULL
    ),
    mappable_rast AS (
        SELECT DISTINCT gr.rast
        FROM genome_rast gr
        INNER JOIN {namespace}.term_association ta
            ON gr.rast = ta.subject
        WHERE ta.object LIKE 'seed.reaction:%'
    )
    SELECT 
        (SELECT COUNT(*) FROM genome_rast) as total_rast_roles,
        (SELECT COUNT(*) FROM mappable_rast) as mappable_rast_roles
    """
    stats = spark.sql(stats_query).collect()[0]
    print(f"\nMapping success rate:")
    print(f"  - Total unique RAST roles: {stats['total_rast_roles']}")
    print(f"  - Roles that map to reactions: {stats['mappable_rast_roles']}")
    print(f"  - Success rate: {stats['mappable_rast_roles']/stats['total_rast_roles']*100:.1f}%")

time_query("Verify RAST to Role Connection", verify_rast_to_role_connection)


Executing: Verify RAST to Role Connection


RAST roles that successfully map to reactions:


,role_string,reaction_count
0,3-hydroxyacyl-[acyl-carrier-protein] dehydrata...,20
1,Peptidase B (EC 3.4.11.23),19
2,Cytosol aminopeptidase PepA (EC 3.4.11.1),19
3,"Aminopeptidase YpdF (MP-, MA-, MS-, AP-, NP- s...",19
4,Membrane alanine aminopeptidase N (EC 3.4.11.2),18
5,Uridine kinase (EC 2.7.1.48),18
6,Methionine aminopeptidase (EC 3.4.11.18),17
7,Glycerol-3-phosphate acyltransferase (EC 2.3.1...,16
8,Phosphatidylserine decarboxylase (EC 4.1.1.65),13
9,Phosphatidate cytidylyltransferase (EC 2.7.7.41),13



Mapping success rate:
  - Total unique RAST roles: 3832
  - Roles that map to reactions: 808
  - Success rate: 21.1%

Query execution time: 4.94 seconds


### 6. Final Query 0: Genome Feature to Reaction Mapping

Now let's put it all together - this is the complete query that maps genome features to reactions:

In [7]:
def query_genome_reactions():
    query = f"""
    WITH genome_features AS (
        -- Step 1: Get features with RAST annotations from our genome
        SELECT 
            f.genome_id,
            f.feature_id,
            f.rast
        FROM {namespace}.feature_annotation f
        WHERE f.genome_id = '562.61239'
        AND f.rast IS NOT NULL
    ),
    feature_reactions AS (
        -- Step 2: Map RAST roles to SEED reactions via term_association
        SELECT DISTINCT
            gf.genome_id,
            gf.feature_id,
            gf.rast,
            ta.object as seed_reaction
        FROM genome_features gf
        INNER JOIN {namespace}.term_association ta
            ON gf.rast = ta.subject  -- This is the key join!
        WHERE ta.object LIKE 'seed.reaction:%'
    ),
    reaction_names AS (
        -- Step 3: Get human-readable reaction names from statements
        SELECT 
            subject as reaction_id,
            value as reaction_name
        FROM {namespace}.statements
        WHERE predicate = 'rdfs:label'
        AND subject LIKE 'seed.reaction:%'
    )
    -- Step 4: Combine everything
    SELECT 
        fr.genome_id,
        fr.feature_id,
        fr.rast,
        fr.seed_reaction,
        rn.reaction_name
    FROM feature_reactions fr
    LEFT JOIN reaction_names rn ON fr.seed_reaction = rn.reaction_id
    ORDER BY fr.genome_id, fr.feature_id
    """
    
    df = spark.sql(query).toPandas()
    print(f"Genome features mapped to their catalyzed reactions:")
    display(df.head(20))
    print(f"\nTotal features with reaction mappings shown: {len(df)}")

time_query("Complete Genome Feature to Reaction Mapping", query_genome_reactions)


Executing: Complete Genome Feature to Reaction Mapping


Genome features mapped to their catalyzed reactions:


,genome_id,feature_id,rast,seed_reaction,reaction_name
0,562.61239,562.61239_1000,Malate synthase G (EC 2.3.3.9),seed.reaction:rxn00330,acetyl-CoA:glyoxylate C-acetyltransferase (thi...
1,562.61239,562.61239_1001,Glycolate permease,seed.reaction:rxn05470,"glycolate transport via proton symport, revers..."
2,562.61239,562.61239_1028,L-asparaginase (EC 3.5.1.1),seed.reaction:rxn00342,L-Asparagine amidohydrolase
3,562.61239,562.61239_1031,"Nucleoside 5-triphosphatase RdgB (dHAPTP, dITP...",seed.reaction:rxn00514,Inosine 5'-triphosphate pyrophosphohydrolase
4,562.61239,562.61239_1031,"Nucleoside 5-triphosphatase RdgB (dHAPTP, dITP...",seed.reaction:rxn02518,2'-Deoxyinosine-5'-triphosphate pyrophosphohyd...
5,562.61239,562.61239_1031,"Nucleoside 5-triphosphatase RdgB (dHAPTP, dITP...",seed.reaction:rxn01962,XTP pyrophosphohydrolase
6,562.61239,562.61239_1038,Glutathione synthetase (EC 6.3.2.3),seed.reaction:rxn00351,gamma-L-glutamyl-L-cysteine:glycine ligase (AD...
7,562.61239,562.61239_1043,S-adenosylmethionine synthetase (EC 2.5.1.6),seed.reaction:rxn03264,ATP:L-selenomethione S-adenosyltransferase
8,562.61239,562.61239_1043,S-adenosylmethionine synthetase (EC 2.5.1.6),seed.reaction:rxn00126,ATP:L-methionine S-adenosyltransferase
9,562.61239,562.61239_1047,Biosynthetic arginine decarboxylase (EC 4.1.1.19),seed.reaction:rxn00405,L-arginine carboxy-lyase (agmatine-forming)



Total features with reaction mappings shown: 1627

Query execution time: 5.25 seconds


---

## Part 2: Extra Queries - Comparative Analysis of 50 E. coli Strains

Now let's explore the diversity across all 50 E. coli genomes using multi-table queries.

### 1. Core vs Accessory Genes: What's Universal vs Strain-Specific?

This query analyzes the pangenome structure by examining which genes (UniRef clusters) are shared across all 50 strains (core genes) versus those found in only some strains (accessory genes). Understanding the core genome helps identify essential functions for E. coli survival, while accessory genes reveal the evolutionary adaptations and niche-specific capabilities.

In [12]:
def analyze_core_accessory_genes():
    query = f"""
    WITH gene_distribution AS (
        -- Count how many genomes have each UniRef cluster
        SELECT 
            bakta_uniref,
            bakta_product,
            COUNT(DISTINCT genome_id) as genome_count,
            COLLECT_SET(genome_id) as genome_list
        FROM {namespace}.feature_annotation
        WHERE bakta_uniref IS NOT NULL
        GROUP BY bakta_uniref, bakta_product
    ),
    categorized_genes AS (
        SELECT 
            *,
            CASE 
                WHEN genome_count = 50 THEN 'Core'
                WHEN genome_count >= 45 THEN 'Soft-core'
                WHEN genome_count >= 25 THEN 'Shell'
                WHEN genome_count >= 2 THEN 'Cloud'
                ELSE 'Unique'
            END as gene_category
        FROM gene_distribution
    )
    SELECT 
        gene_category,
        COUNT(*) as gene_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM categorized_genes
    GROUP BY gene_category
    ORDER BY 
        CASE gene_category
            WHEN 'Core' THEN 1
            WHEN 'Soft-core' THEN 2
            WHEN 'Shell' THEN 3
            WHEN 'Cloud' THEN 4
            WHEN 'Unique' THEN 5
        END
    """
    
    df = spark.sql(query).toPandas()
    print(f"E. coli pangenome structure across 50 strains:")
    display(df)
    
    # Show examples of core genes
    core_examples_query = f"""
    SELECT DISTINCT
        bakta_uniref,
        bakta_product,
        bakta_gene
    FROM {namespace}.feature_annotation
    WHERE bakta_uniref IN (
        SELECT bakta_uniref
        FROM {namespace}.feature_annotation
        WHERE bakta_uniref IS NOT NULL
        GROUP BY bakta_uniref
        HAVING COUNT(DISTINCT genome_id) = 50
    )
    LIMIT 10
    """
    
    core_df = spark.sql(core_examples_query).toPandas()
    print(f"\nExamples of core genes (present in all 50 strains):")
    display(core_df)

time_query("Core vs Accessory Gene Analysis", analyze_core_accessory_genes)


Executing: Core vs Accessory Gene Analysis


[Stage 152:==========================================>              (3 + 1) / 4]

E. coli pangenome structure across 50 strains:


,gene_category,gene_count,percentage
0,Core,737,3.18
1,Soft-core,2240,9.67
2,Shell,1011,4.36
3,Cloud,8272,35.69
4,Unique,10916,47.10



Examples of core genes (present in all 50 strains):


,bakta_uniref,bakta_product,bakta_gene
0,"UniRef50_P0AEK4,UniRef90_P0AEK4",enoyl-ACP reductase FabI,fabI
1,"UniRef50_P23524,UniRef90_P23524",glycerate 2-kinase,garK
2,"UniRef50_P0A6T3,UniRef90_P0A6T3",galactokinase,galK
3,"UniRef50_P27249,UniRef90_Q3Z5J1",bifunctional uridylyltransferase/uridylyl-remo...,glnD
4,"UniRef50_A0A4V2JFY6,UniRef90_UPI00193345D9",flagellar type III secretion system pore prote...,fliP
5,"UniRef50_P21507,UniRef90_P21507",ATP-dependent RNA helicase SrmB,srmB
6,"UniRef50_P77301,UniRef90_P77301",Uncharacterized protein YbaP,ybaP
7,"UniRef50_P64506,UniRef90_P64506",Uncharacterized protein YebY,yebY
8,"UniRef50_P37902,UniRef90_P37902",glutamate/aspartate ABC transporter substrate-...,gltI
9,"UniRef50_P0A8F4,UniRef90_P0A8F4",uridine kinase,udk



Query execution time: 8.31 seconds


### 2. Functional Diversity: EC Number Distribution Across Strains

This query examines the distribution of enzyme commission (EC) numbers across strains to understand functional diversity. By categorizing enzymes as universal, conserved, or variable, we can identify which metabolic capabilities are essential versus those that provide strain-specific advantages.

In [13]:
def analyze_ec_diversity():
    query = f"""
    WITH strain_ec_profiles AS (
        -- Get EC number profiles for each strain
        SELECT 
            f.genome_id,
            f.bakta_ec,
            s.value as ec_name
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s
            ON CONCAT('EC:', f.bakta_ec) = s.subject
            AND s.predicate = 'rdfs:label'
        WHERE f.bakta_ec IS NOT NULL
    ),
    ec_distribution AS (
        -- Count how many strains have each EC number
        SELECT 
            bakta_ec,
            MAX(ec_name) as ec_name,
            COUNT(DISTINCT genome_id) as strain_count,
            ROUND(COUNT(DISTINCT genome_id) * 100.0 / 50, 1) as prevalence_pct
        FROM strain_ec_profiles
        GROUP BY bakta_ec
    ),
    categorized_ec AS (
        SELECT 
            *,
            CASE 
                WHEN strain_count = 50 THEN 'Universal'
                WHEN strain_count >= 40 THEN 'Highly conserved'
                WHEN strain_count >= 25 THEN 'Common'
                WHEN strain_count >= 10 THEN 'Variable'
                ELSE 'Rare'
            END as conservation_category
        FROM ec_distribution
    )
    SELECT 
        conservation_category,
        COUNT(*) as ec_count,
        MIN(strain_count) as min_strains,
        MAX(strain_count) as max_strains,
        ROUND(AVG(prevalence_pct), 1) as avg_prevalence_pct
    FROM categorized_ec
    GROUP BY conservation_category
    ORDER BY max_strains DESC
    """
    
    df = spark.sql(query).toPandas()
    print(f"EC number conservation across 50 E. coli strains:")
    display(df)
    
    # Show variable EC functions
    variable_ec_query = f"""
    WITH ec_counts AS (
        SELECT 
            f.bakta_ec,
            s.value as ec_name,
            COUNT(DISTINCT f.genome_id) as strain_count
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s
            ON CONCAT('EC:', f.bakta_ec) = s.subject
            AND s.predicate = 'rdfs:label'
        WHERE f.bakta_ec IS NOT NULL
        GROUP BY f.bakta_ec, s.value
    )
    SELECT * FROM ec_counts
    WHERE strain_count BETWEEN 10 AND 40
    ORDER BY strain_count DESC
    LIMIT 15
    """
    
    variable_df = spark.sql(variable_ec_query).toPandas()
    print(f"\nVariable enzymatic functions (present in 10-40 strains):")
    display(variable_df)

time_query("EC Number Diversity Analysis", analyze_ec_diversity)


Executing: EC Number Diversity Analysis
EC number conservation across 50 E. coli strains:


,conservation_category,ec_count,min_strains,max_strains,avg_prevalence_pct
0,Universal,478,50,50,100.0
1,Highly conserved,637,40,49,95.8
2,Common,39,26,39,68.4
3,Variable,30,10,23,31.0
4,Rare,138,1,9,6.6


[Stage 212:==================================>                      (3 + 2) / 5]


Variable enzymatic functions (present in 10-40 strains):


,bakta_ec,ec_name,strain_count
0,"2.5.1.-,2.9.1.3",None,40
1,1.6.99.-,None,39
2,1.4.3.21,primary-amine oxidase,39
3,2.3.2.2,gamma-glutamyltransferase,39
4,2.4.1.352,glucosylglycerate phosphorylase,39
5,1.2.99.6,carboxylate reductase,39
6,3.5.1.42,nicotinamide-nucleotide amidase,39
7,2.7.1.195,phosphotransferase,39
8,6.2.1.30,phenylacetate--CoA ligase,39
9,5.3.3.10,5-carboxymethyl-2-hydroxymuconate Delta-isomerase,38



Query execution time: 11.61 seconds


### 3. Metabolic Capability Differences: Reaction Sets Between Strains

This query compares the total number of metabolic reactions each strain can perform by mapping RAST annotations to SEED reactions. Strains with more reactions have broader metabolic capabilities, potentially allowing them to thrive in more diverse environments or utilize a wider range of nutrients.

In [14]:
def compare_metabolic_capabilities():
    query = f"""
    WITH strain_reactions AS (
        -- Map each strain to its set of reactions (optimized with early filtering)
        SELECT DISTINCT
            f.genome_id,
            ta.object as reaction_id
        FROM {namespace}.feature_annotation f
        INNER JOIN {namespace}.term_association ta
            ON f.rast = ta.subject
        WHERE f.rast IS NOT NULL
        AND ta.object LIKE 'seed.reaction:%'
        AND ta.predicate = 'RO:0002327'
    ),
    strain_reaction_counts AS (
        -- Count reactions per strain
        SELECT 
            genome_id,
            COUNT(DISTINCT reaction_id) as reaction_count
        FROM strain_reactions
        GROUP BY genome_id
    ),
    stats AS (
        SELECT 
            MIN(reaction_count) as min_reactions,
            MAX(reaction_count) as max_reactions,
            AVG(reaction_count) as avg_reactions,
            STDDEV(reaction_count) as std_reactions
        FROM strain_reaction_counts
    )
    SELECT 
        'Metabolic Capacity Statistics' as metric,
        min_reactions,
        max_reactions,
        ROUND(avg_reactions, 1) as avg_reactions,
        ROUND(std_reactions, 1) as std_reactions,
        max_reactions - min_reactions as reaction_range
    FROM stats
    """
    
    df = spark.sql(query).toPandas()
    print(f"Metabolic reaction capacity across strains:")
    display(df)
    
    # Show strains with extreme metabolic capacities (simplified query)
    extremes_query = f"""
    WITH strain_counts AS (
        SELECT 
            f.genome_id,
            COUNT(DISTINCT ta.object) as reaction_count
        FROM {namespace}.feature_annotation f
        INNER JOIN {namespace}.term_association ta
            ON f.rast = ta.subject
        WHERE f.rast IS NOT NULL
        AND ta.object LIKE 'seed.reaction:%'
        AND ta.predicate = 'RO:0002327'
        GROUP BY f.genome_id
    ),
    ranked_strains AS (
        SELECT 
            genome_id,
            reaction_count,
            ROW_NUMBER() OVER (ORDER BY reaction_count DESC) as rank_desc,
            ROW_NUMBER() OVER (ORDER BY reaction_count ASC) as rank_asc
        FROM strain_counts
    )
    SELECT 
        genome_id,
        reaction_count,
        CASE 
            WHEN rank_desc <= 5 THEN 'Highest capacity'
            WHEN rank_asc <= 5 THEN 'Lowest capacity'
        END as category
    FROM ranked_strains
    WHERE rank_desc <= 5 OR rank_asc <= 5
    ORDER BY reaction_count DESC
    """
    
    extremes_df = spark.sql(extremes_query).toPandas()
    print(f"\nStrains with extreme metabolic capacities:")
    display(extremes_df)

time_query("Metabolic Capability Comparison", compare_metabolic_capabilities)


Executing: Metabolic Capability Comparison


Metabolic reaction capacity across strains:


,metric,min_reactions,max_reactions,avg_reactions,std_reactions,reaction_range
0,Metabolic Capacity Statistics,1027,1099,1082.5,14.4,72



Strains with extreme metabolic capacities:


,genome_id,reaction_count,category
0,562.61119,1099,Highest capacity
1,562.61097,1099,Highest capacity
2,562.55859,1098,Highest capacity
3,562.55864,1096,Highest capacity
4,562.61192,1096,Highest capacity
5,562.61169,1065,Lowest capacity
6,562.61073,1062,Lowest capacity
7,562.55868,1056,Lowest capacity
8,562.61106,1042,Lowest capacity
9,562.55845,1027,Lowest capacity



Query execution time: 5.44 seconds


### 4. Taxonomic Clustering by Functional Profiles

This query profiles each strain by its enzyme class distribution (oxidoreductases, transferases, etc.) to identify functional similarities. Strains with similar enzyme profiles likely have similar metabolic capabilities and may occupy similar ecological niches.

In [15]:
def analyze_taxonomic_functional_clusters():
    query = f"""
    WITH strain_taxonomy AS (
        -- Get taxonomic info for each strain
        SELECT DISTINCT
            f.genome_id,
            f.genome_taxa,
            s.value as strain_name
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s
            ON f.genome_taxa = s.subject AND s.predicate = 'rdfs:label'
    ),
    strain_functions AS (
        -- Get functional profile (EC numbers) for each strain
        SELECT 
            genome_id,
            COUNT(DISTINCT bakta_ec) as ec_count,
            COUNT(DISTINCT bakta_go) as go_count,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '1.%' THEN bakta_ec END) as oxidoreductases,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '2.%' THEN bakta_ec END) as transferases,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '3.%' THEN bakta_ec END) as hydrolases,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '4.%' THEN bakta_ec END) as lyases,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '5.%' THEN bakta_ec END) as isomerases,
            COUNT(DISTINCT CASE WHEN bakta_ec LIKE '6.%' THEN bakta_ec END) as ligases
        FROM {namespace}.feature_annotation
        WHERE bakta_ec IS NOT NULL
        GROUP BY genome_id
    )
    SELECT 
        st.genome_id,
        st.strain_name,
        sf.ec_count,
        sf.go_count,
        sf.oxidoreductases,
        sf.transferases,
        sf.hydrolases,
        sf.lyases,
        sf.isomerases,
        sf.ligases,
        ROUND(sf.transferases * 100.0 / sf.ec_count, 1) as transferase_pct,
        ROUND(sf.hydrolases * 100.0 / sf.ec_count, 1) as hydrolase_pct
    FROM strain_taxonomy st
    JOIN strain_functions sf ON st.genome_id = sf.genome_id
    ORDER BY sf.ec_count DESC
    LIMIT 10
    """
    
    df = spark.sql(query).toPandas()
    print(f"Functional enzyme profiles by strain:")
    display(df)

time_query("Taxonomic-Functional Clustering", analyze_taxonomic_functional_clusters)


Executing: Taxonomic-Functional Clustering


[Stage 268:==========================================>              (3 + 1) / 4]

Functional enzyme profiles by strain:


,genome_id,strain_name,ec_count,go_count,oxidoreductases,transferases,hydrolases,lyases,isomerases,ligases,transferase_pct,hydrolase_pct
0,562.61071,None,1151,1498,219,378,245,118,72,71,32.8,21.3
1,562.61115,None,1151,1497,223,372,246,119,72,71,32.3,21.4
2,562.61207,None,1151,1497,223,375,247,116,71,70,32.6,21.5
3,562.61119,None,1150,1501,227,369,247,118,72,70,32.1,21.5
4,562.61195,None,1149,1502,221,369,250,118,73,69,32.1,21.8
5,562.61198,None,1148,1493,219,371,248,120,73,70,32.3,21.6
6,562.61240,None,1146,1497,220,370,248,118,73,71,32.3,21.6
7,562.61097,None,1143,1499,221,368,243,122,72,70,32.2,21.3
8,562.61163,None,1143,1499,220,367,246,121,72,70,32.1,21.5
9,562.61183,None,1142,1480,221,365,246,122,70,71,32.0,21.5



Query execution time: 7.95 seconds


### 5. Pathway Completeness Analysis

This query examines the completeness of essential metabolic pathways across strains, using glycolysis as an example. Pathway completeness analysis helps identify strains with potential metabolic deficiencies or alternative pathways, which could indicate adaptation to specific environments or nutritional requirements.

In [16]:
def analyze_pathway_completeness():
    query = f"""
    WITH glycolysis_reactions AS (
        -- Search for glycolysis reactions by name patterns
        SELECT DISTINCT 
            subject as reaction_id,
            value as reaction_name
        FROM {namespace}.statements
        WHERE subject LIKE 'seed.reaction:%'
        AND predicate = 'rdfs:label'
        AND (
            LOWER(value) LIKE '%glucose%phosphate isomerase%'
            OR LOWER(value) LIKE '%phosphofructokinase%'
            OR LOWER(value) LIKE '%fructose%bisphosphate aldolase%'
            OR LOWER(value) LIKE '%glyceraldehyde%3%phosphate dehydrogenase%'
            OR LOWER(value) LIKE '%phosphoglycerate kinase%'
            OR LOWER(value) LIKE '%phosphoglycerate mutase%'
            OR LOWER(value) LIKE '%enolase%'
            OR LOWER(value) LIKE '%pyruvate kinase%'
        )
    ),
    strain_glycolysis_coverage AS (
        -- Check which strains have which glycolysis reactions
        SELECT 
            f.genome_id,
            COUNT(DISTINCT gr.reaction_id) as glycolysis_reactions_present
        FROM {namespace}.feature_annotation f
        INNER JOIN {namespace}.term_association ta ON f.rast = ta.subject
        INNER JOIN glycolysis_reactions gr ON ta.object = gr.reaction_id
        WHERE f.rast IS NOT NULL
        GROUP BY f.genome_id
    ),
    all_strains AS (
        -- Include all strains, even those with 0 reactions
        SELECT DISTINCT genome_id FROM {namespace}.feature_annotation
    ),
    strain_coverage_complete AS (
        SELECT 
            a.genome_id,
            COALESCE(s.glycolysis_reactions_present, 0) as reactions_present,
            (SELECT COUNT(*) FROM glycolysis_reactions) as total_reactions
        FROM all_strains a
        LEFT JOIN strain_glycolysis_coverage s ON a.genome_id = s.genome_id
    )
    SELECT 
        CASE 
            WHEN reactions_present >= 7 THEN 'Complete (7-8 reactions)'
            WHEN reactions_present >= 5 THEN 'Nearly complete (5-6)'
            WHEN reactions_present >= 3 THEN 'Partial (3-4)'
            WHEN reactions_present >= 1 THEN 'Minimal (1-2)'
            ELSE 'None detected'
        END as completeness_category,
        COUNT(*) as strain_count,
        ROUND(COUNT(*) * 100.0 / 50, 1) as percentage
    FROM strain_coverage_complete
    GROUP BY completeness_category
    ORDER BY 
        CASE completeness_category
            WHEN 'Complete (7-8 reactions)' THEN 1
            WHEN 'Nearly complete (5-6)' THEN 2
            WHEN 'Partial (3-4)' THEN 3
            WHEN 'Minimal (1-2)' THEN 4
            ELSE 5
        END
    """
    
    df = spark.sql(query).toPandas()
    print(f"Glycolysis pathway completeness across strains:")
    display(df)
    
    # Show which reactions were found
    reactions_found_query = f"""
    WITH glycolysis_reactions AS (
        SELECT DISTINCT 
            subject as reaction_id,
            value as reaction_name
        FROM {namespace}.statements
        WHERE subject LIKE 'seed.reaction:%'
        AND predicate = 'rdfs:label'
        AND (
            LOWER(value) LIKE '%glucose%phosphate isomerase%'
            OR LOWER(value) LIKE '%phosphofructokinase%'
            OR LOWER(value) LIKE '%fructose%bisphosphate aldolase%'
            OR LOWER(value) LIKE '%glyceraldehyde%3%phosphate dehydrogenase%'
            OR LOWER(value) LIKE '%phosphoglycerate kinase%'
            OR LOWER(value) LIKE '%phosphoglycerate mutase%'
            OR LOWER(value) LIKE '%enolase%'
            OR LOWER(value) LIKE '%pyruvate kinase%'
        )
    )
    SELECT * FROM glycolysis_reactions
    ORDER BY reaction_name
    """
    
    reactions_df = spark.sql(reactions_found_query).toPandas()
    print(f"\nGlycolysis reactions found in SEED:")
    display(reactions_df)

time_query("Pathway Completeness Analysis", analyze_pathway_completeness)


Executing: Pathway Completeness Analysis
Glycolysis pathway completeness across strains:


,completeness_category,strain_count,percentage
0,None detected,50,100.0



Glycolysis reactions found in SEED:


,reaction_id,reaction_name
0,seed.reaction:rxn33754,6-phosphofructokinase
1,seed.reaction:rxn33841,Fructose Bisphosphate Aldolase
2,seed.reaction:rxn30127,Fructose-bisphosphate aldolase
3,seed.reaction:rxn33838,Glucose 6 Phosphate Isomerase
4,seed.reaction:rxn30182,Glucose-6-phosphate isomerase
5,seed.reaction:rxn33486,Glucose-6-phosphate isomerase
6,seed.reaction:rxn30181,Glucose-6-phosphate isomerase
7,seed.reaction:rxn33210,Glyceraldehyde 3-phosphate dehydrogenase (phos...
8,seed.reaction:rxn33274,Glyceraldehyde 3-phosphate dehydrogenase (phos...
9,seed.reaction:rxn33275,Glyceraldehyde 3-phosphate dehydrogenase (phos...



Query execution time: 5.71 seconds


### 6. Unique Functional Features by Strain

This query identifies functions (RAST roles) that are unique to individual strains, revealing strain-specific metabolic capabilities. These unique features may represent recent acquisitions through horizontal gene transfer or specialized adaptations to particular environments.

In [17]:
def find_unique_strain_features():
    query = f"""
    WITH feature_distribution AS (
        -- Find features unique to single strains
        SELECT 
            rast,
            COUNT(DISTINCT genome_id) as strain_count,
            COLLECT_SET(genome_id)[0] as unique_to_genome
        FROM {namespace}.feature_annotation
        WHERE rast IS NOT NULL
        GROUP BY rast
        HAVING COUNT(DISTINCT genome_id) = 1
    ),
    unique_features_with_reactions AS (
        -- See if these unique features have known reactions
        SELECT 
            fd.unique_to_genome,
            fd.rast,
            ta.object as reaction_id,
            s1.value as reaction_name,
            s2.value as strain_name
        FROM feature_distribution fd
        LEFT JOIN {namespace}.term_association ta ON fd.rast = ta.subject
        LEFT JOIN {namespace}.statements s1 
            ON ta.object = s1.subject AND s1.predicate = 'rdfs:label'
        LEFT JOIN {namespace}.feature_annotation fa ON fd.unique_to_genome = fa.genome_id
        LEFT JOIN {namespace}.statements s2 
            ON fa.genome_taxa = s2.subject AND s2.predicate = 'rdfs:label'
        WHERE ta.object LIKE 'seed.reaction:%'
    ),
    strain_unique_counts AS (
        SELECT 
            unique_to_genome,
            MAX(strain_name) as strain_name,
            COUNT(DISTINCT rast) as unique_functions,
            COUNT(DISTINCT reaction_id) as unique_reactions
        FROM unique_features_with_reactions
        GROUP BY unique_to_genome
    )
    SELECT * FROM strain_unique_counts
    WHERE unique_reactions > 0
    ORDER BY unique_reactions DESC
    LIMIT 15
    """
    
    df = spark.sql(query).toPandas()
    print(f"Strains with unique metabolic capabilities:")
    display(df)
    
    # Show examples of unique functions
    examples_query = f"""
    WITH unique_features AS (
        SELECT 
            rast,
            COLLECT_SET(genome_id)[0] as genome_id
        FROM {namespace}.feature_annotation
        WHERE rast IS NOT NULL
        GROUP BY rast
        HAVING COUNT(DISTINCT genome_id) = 1
    )
    SELECT 
        uf.genome_id,
        uf.rast as unique_function,
        ta.object as reaction_id,
        s.value as reaction_name
    FROM unique_features uf
    INNER JOIN {namespace}.term_association ta ON uf.rast = ta.subject
    LEFT JOIN {namespace}.statements s 
        ON ta.object = s.subject AND s.predicate = 'rdfs:label'
    WHERE ta.object LIKE 'seed.reaction:%'
    LIMIT 10
    """
    
    examples_df = spark.sql(examples_query).toPandas()
    print(f"\nExamples of strain-specific functions and reactions:")
    display(examples_df)

time_query("Unique Strain Features Analysis", find_unique_strain_features)


Executing: Unique Strain Features Analysis


[Stage 329:==========================================>              (3 + 1) / 4]

Strains with unique metabolic capabilities:


,unique_to_genome,strain_name,unique_functions,unique_reactions
0,562.61106,None,4,4
1,562.55868,None,1,2
2,562.61167,None,2,2
3,562.61179,None,2,2
4,562.61197,None,2,2
5,562.8507,None,2,2
6,562.55577,None,1,1
7,562.55845,None,1,1
8,562.61175,None,1,1


[Stage 355:============================>                            (2 + 2) / 4]


Examples of strain-specific functions and reactions:


,genome_id,unique_function,reaction_id,reaction_name
0,562.61179,5-methylthioribose kinase (EC 2.7.1.100),seed.reaction:rxn02894,ATP:S5-methyl-5-thio-D-ribose 1-phosphotransfe...
1,562.61175,Beta-ketoadipate enol-lactone hydrolase (EC 3....,seed.reaction:rxn02144,4-carboxymethylbut-3-en-4-olide enol-lactonohy...
2,562.61167,Maleate cis-trans isomerase (EC 5.2.1.1),seed.reaction:rxn00803,Maleate cis-trans-isomerase
3,562.8507,Shikimate 5-dehydrogenase I gamma (EC 1.1.1.25),seed.reaction:rxn01740,Shikimate:NADP+ 3-oxidoreductase
4,562.61106,3-dehydroquinate dehydratase II (EC 4.2.1.10),seed.reaction:rxn02213,3-Dehydroquinate hydro-lyase
5,562.55868,4-hydroxyphenylpyruvate dioxygenase (EC 1.13.1...,seed.reaction:rxn01827,4-Hydroxyphenylpyruvate:oxygen oxidoreductase ...
6,562.55868,4-hydroxyphenylpyruvate dioxygenase (EC 1.13.1...,seed.reaction:rxn00999,Phenylpyruvate:oxygen oxidoreductase (hydroxyl...
7,562.8507,Arginine decarboxylase (EC 4.1.1.19),seed.reaction:rxn00405,L-arginine carboxy-lyase (agmatine-forming)
8,562.61197,D-arabinitol 4-dehydrogenase (EC 1.1.1.11),seed.reaction:rxn03883,D-arabinitol:NAD 4-oxidoreductase
9,562.55577,D-arabino-3-hexulose 6-phosphate formaldehyde-...,seed.reaction:rxn03643,D-arabino-hex-3-ulose-6-phosphate formaldehyde...



Query execution time: 13.97 seconds


### 7. Conservation Analysis: Most and Least Conserved Functions

This query analyzes GO term conservation to identify molecular functions that are universally conserved versus those that are rare. Highly conserved functions represent the core cellular machinery, while rarely conserved functions may indicate specialized adaptations or recent evolutionary innovations.

### 8. GO Term Hierarchy Analysis

This query explores the parent-child relationships in GO ontology by analyzing which GO terms have the most children and how deep the hierarchy goes. Understanding GO hierarchy helps reveal functional specialization patterns and the granularity of annotations across strains.

In [18]:
def analyze_go_hierarchy():
    query = f"""
    WITH go_relationships AS (
        -- Get parent-child relationships from statements
        SELECT 
            s1.subject as child_go,
            s1.object as parent_go,
            s2.value as child_name,
            s3.value as parent_name
        FROM {namespace}.statements s1
        LEFT JOIN {namespace}.statements s2 
            ON s1.subject = s2.subject AND s2.predicate = 'rdfs:label'
        LEFT JOIN {namespace}.statements s3 
            ON s1.object = s3.subject AND s3.predicate = 'rdfs:label'
        WHERE s1.predicate = 'rdfs:subClassOf'
        AND s1.subject LIKE 'GO:%'
        AND s1.object LIKE 'GO:%'
    ),
    parent_child_counts AS (
        -- Count children for each parent GO term
        SELECT 
            parent_go,
            MAX(parent_name) as parent_name,
            COUNT(DISTINCT child_go) as child_count
        FROM go_relationships
        GROUP BY parent_go
    ),
    go_in_annotations AS (
        -- Find which GO terms are actually used in our annotations
        SELECT DISTINCT bakta_go as go_term
        FROM {namespace}.feature_annotation
        WHERE bakta_go IS NOT NULL
    )
    SELECT 
        pcc.parent_go,
        pcc.parent_name,
        pcc.child_count,
        CASE WHEN gia.go_term IS NOT NULL THEN 'Used in annotations' ELSE 'Not used' END as usage_status
    FROM parent_child_counts pcc
    LEFT JOIN go_in_annotations gia ON pcc.parent_go = gia.go_term
    WHERE pcc.child_count >= 10
    ORDER BY pcc.child_count DESC
    LIMIT 10
    """
    
    df = spark.sql(query).toPandas()
    print(f"GO terms with the most children in the hierarchy:")
    display(df)
    
    # Analyze depth of GO terms used in annotations
    depth_query = f"""
    WITH RECURSIVE go_depth AS (
        -- Base case: root GO terms (no parents)
        SELECT 
            subject as go_term,
            0 as depth
        FROM {namespace}.statements
        WHERE subject LIKE 'GO:%'
        AND subject NOT IN (
            SELECT subject 
            FROM {namespace}.statements 
            WHERE predicate = 'rdfs:subClassOf' 
            AND object LIKE 'GO:%'
        )
        
        UNION ALL
        
        -- Recursive case: children of current level
        SELECT 
            s.subject as go_term,
            gd.depth + 1 as depth
        FROM {namespace}.statements s
        JOIN go_depth gd ON s.object = gd.go_term
        WHERE s.predicate = 'rdfs:subClassOf'
        AND s.subject LIKE 'GO:%'
    ),
    annotated_go_depth AS (
        SELECT 
            f.bakta_go,
            MAX(gd.depth) as max_depth
        FROM {namespace}.feature_annotation f
        JOIN go_depth gd ON f.bakta_go = gd.go_term
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.bakta_go
    )
    SELECT 
        CASE 
            WHEN max_depth <= 3 THEN 'Shallow (0-3 levels)'
            WHEN max_depth <= 6 THEN 'Medium (4-6 levels)'
            WHEN max_depth <= 9 THEN 'Deep (7-9 levels)'
            ELSE 'Very deep (10+ levels)'
        END as depth_category,
        COUNT(*) as go_term_count,
        AVG(max_depth) as avg_depth
    FROM annotated_go_depth
    GROUP BY depth_category
    ORDER BY avg_depth
    """
    
    # Note: Recursive queries might not be supported, so let's use a simpler approach
    simple_depth_query = f"""
    WITH go_with_parents AS (
        SELECT 
            f.bakta_go,
            COUNT(DISTINCT s.object) as parent_count
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s 
            ON f.bakta_go = s.subject 
            AND s.predicate = 'rdfs:subClassOf'
            AND s.object LIKE 'GO:%'
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.bakta_go
    )
    SELECT 
        CASE 
            WHEN parent_count = 0 THEN 'Root terms'
            WHEN parent_count = 1 THEN 'Single parent'
            WHEN parent_count = 2 THEN 'Two parents'
            ELSE 'Multiple parents (3+)'
        END as hierarchy_type,
        COUNT(*) as go_term_count
    FROM go_with_parents
    GROUP BY hierarchy_type
    ORDER BY go_term_count DESC
    """
    
    hierarchy_df = spark.sql(simple_depth_query).toPandas()
    print(f"\nGO term hierarchy patterns in annotations:")
    display(hierarchy_df)

time_query("GO Term Hierarchy Analysis", analyze_go_hierarchy)


Executing: GO Term Hierarchy Analysis


GO terms with the most children in the hierarchy:


,parent_go,parent_name,child_count,usage_status
0,GO:0110165,cellular anatomical structure,434,Not used
1,GO:0016616,"oxidoreductase activity, acting on the CH-OH g...",282,Not used
2,GO:0032991,protein-containing complex,280,Not used
3,GO:0048856,anatomical structure development,202,Not used
4,GO:0140513,nuclear protein-containing complex,172,Not used
5,GO:0098797,plasma membrane protein complex,168,Not used
6,GO:0016709,"oxidoreductase activity, acting on paired dono...",162,Not used
7,GO:0003006,developmental process involved in reproduction,151,Not used
8,GO:1901700,response to oxygen-containing compound,142,Not used
9,GO:0051241,negative regulation of multicellular organisma...,130,Not used



GO term hierarchy patterns in annotations:


,hierarchy_type,go_term_count
0,Root terms,3080
1,Single parent,137
2,Two parents,37
3,Multiple parents (3+),17



Query execution time: 9.34 seconds


In [19]:
def analyze_function_conservation():
    query = f"""
    WITH go_conservation AS (
        -- Analyze GO term conservation
        SELECT 
            f.bakta_go as go_term,
            s.value as go_name,
            COUNT(DISTINCT f.genome_id) as strain_count,
            COUNT(*) as total_annotations,
            ROUND(COUNT(DISTINCT f.genome_id) * 100.0 / 50, 1) as conservation_pct
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s
            ON f.bakta_go = s.subject AND s.predicate = 'rdfs:label'
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.bakta_go, s.value
    ),
    go_categories AS (
        -- Get GO categories (biological process, molecular function, cellular component)
        SELECT 
            gc.*,
            CASE 
                WHEN gc.go_term LIKE 'GO:00%' THEN 'Molecular Function'
                WHEN gc.go_term LIKE 'GO:000%' THEN 'Biological Process'
                WHEN gc.go_term LIKE 'GO:0005%' THEN 'Cellular Component'
                ELSE 'Other'
            END as go_category
        FROM go_conservation gc
    )
    (SELECT 
        'Most Conserved' as conservation_type,
        go_term,
        go_name,
        go_category,
        strain_count,
        conservation_pct
    FROM go_categories
    WHERE strain_count >= 45
    ORDER BY strain_count DESC, total_annotations DESC
    LIMIT 10)
    
    UNION ALL
    
    (SELECT 
        'Least Conserved' as conservation_type,
        go_term,
        go_name,
        go_category,
        strain_count,
        conservation_pct
    FROM go_categories
    WHERE strain_count <= 5 AND strain_count > 1
    ORDER BY strain_count ASC, total_annotations DESC
    LIMIT 10)
    """
    
    df = spark.sql(query).toPandas()
    print(f"Most and least conserved GO terms across E. coli strains:")
    display(df)
    
    # Summary by GO category
    category_summary_query = f"""
    WITH go_conservation AS (
        SELECT 
            f.bakta_go as go_term,
            COUNT(DISTINCT f.genome_id) as strain_count
        FROM {namespace}.feature_annotation f
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.bakta_go
    ),
    go_categories AS (
        SELECT 
            CASE 
                WHEN go_term LIKE 'GO:00%' THEN 'Molecular Function'
                WHEN go_term LIKE 'GO:000%' THEN 'Biological Process'
                WHEN go_term LIKE 'GO:0005%' THEN 'Cellular Component'
                ELSE 'Other'
            END as go_category,
            strain_count
        FROM go_conservation
    )
    SELECT 
        go_category,
        COUNT(*) as term_count,
        ROUND(AVG(strain_count), 1) as avg_strain_count,
        MIN(strain_count) as min_strains,
        MAX(strain_count) as max_strains
    FROM go_categories
    GROUP BY go_category
    ORDER BY avg_strain_count DESC
    """
    
    summary_df = spark.sql(category_summary_query).toPandas()
    print(f"\nConservation summary by GO category:")
    display(summary_df)

time_query("Function Conservation Analysis", analyze_function_conservation)


Executing: Function Conservation Analysis


Most and least conserved GO terms across E. coli strains:


,conservation_type,go_term,go_name,go_category,strain_count,conservation_pct
0,Most Conserved,GO:0005886,plasma membrane,Molecular Function,50,100.0
1,Most Conserved,GO:0005829,cytosol,Molecular Function,50,100.0
2,Most Conserved,GO:0004803,transposase activity,Molecular Function,50,100.0
3,Most Conserved,GO:0016020,membrane,Molecular Function,50,100.0
4,Most Conserved,"GO:0005886,GO:0022857",None,Molecular Function,50,100.0
5,Most Conserved,GO:0003677,DNA binding,Molecular Function,50,100.0
6,Most Conserved,"GO:0003735,GO:0005840,GO:0006412",None,Molecular Function,50,100.0
7,Most Conserved,"GO:0003700,GO:0006355",None,Molecular Function,50,100.0
8,Most Conserved,"GO:0003677,GO:0006355",None,Molecular Function,50,100.0
9,Most Conserved,GO:0016491,oxidoreductase activity,Molecular Function,50,100.0



Conservation summary by GO category:


,go_category,term_count,avg_strain_count,min_strains,max_strains
0,Molecular Function,3263,37.4,1,50
1,Other,8,26.1,1,49



Query execution time: 13.58 seconds


### 9. GO Enrichment by Strain

This query identifies strain-specific GO term enrichments by comparing the frequency of GO terms in individual strains against the background frequency across all strains. This reveals which biological functions are over-represented in specific strains, suggesting adaptations or specializations.

In [20]:
def analyze_go_enrichment_by_strain():
    query = f"""
    WITH strain_go_counts AS (
        -- Count GO terms per strain
        SELECT 
            f.genome_id,
            f.bakta_go,
            s.value as go_name,
            COUNT(*) as count_in_strain
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s 
            ON f.bakta_go = s.subject AND s.predicate = 'rdfs:label'
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.genome_id, f.bakta_go, s.value
    ),
    global_go_counts AS (
        -- Count GO terms across all strains
        SELECT 
            bakta_go,
            COUNT(*) as global_count,
            COUNT(DISTINCT genome_id) as strains_with_term
        FROM {namespace}.feature_annotation
        WHERE bakta_go IS NOT NULL
        GROUP BY bakta_go
    ),
    strain_totals AS (
        -- Total annotations per strain
        SELECT 
            genome_id,
            COUNT(*) as total_annotations
        FROM {namespace}.feature_annotation
        WHERE bakta_go IS NOT NULL
        GROUP BY genome_id
    ),
    enrichment_analysis AS (
        SELECT 
            sgc.genome_id,
            sgc.bakta_go,
            sgc.go_name,
            sgc.count_in_strain,
            st.total_annotations as strain_total,
            ggc.global_count,
            ggc.strains_with_term,
            -- Calculate enrichment ratio
            (sgc.count_in_strain * 1.0 / st.total_annotations) / 
            (ggc.global_count * 1.0 / (SELECT SUM(total_annotations) FROM strain_totals)) as enrichment_ratio
        FROM strain_go_counts sgc
        JOIN global_go_counts ggc ON sgc.bakta_go = ggc.bakta_go
        JOIN strain_totals st ON sgc.genome_id = st.genome_id
        WHERE ggc.strains_with_term >= 10  -- Only consider GO terms in at least 10 strains
    ),
    top_enrichments AS (
        SELECT *
        FROM enrichment_analysis
        WHERE enrichment_ratio > 2.0  -- At least 2-fold enrichment
        AND count_in_strain >= 5      -- At least 5 occurrences in the strain
    )
    SELECT 
        genome_id,
        COUNT(DISTINCT bakta_go) as enriched_go_terms,
        AVG(enrichment_ratio) as avg_enrichment_ratio,
        MAX(enrichment_ratio) as max_enrichment_ratio
    FROM top_enrichments
    GROUP BY genome_id
    ORDER BY enriched_go_terms DESC
    LIMIT 15
    """
    
    df = spark.sql(query).toPandas()
    print(f"Strains with the most enriched GO terms:")
    display(df)
    
    # Show specific enriched GO terms for top strains
    specific_enrichments_query = f"""
    WITH strain_go_counts AS (
        SELECT 
            f.genome_id,
            f.bakta_go,
            s.value as go_name,
            COUNT(*) as count_in_strain
        FROM {namespace}.feature_annotation f
        LEFT JOIN {namespace}.statements s 
            ON f.bakta_go = s.subject AND s.predicate = 'rdfs:label'
        WHERE f.bakta_go IS NOT NULL
        GROUP BY f.genome_id, f.bakta_go, s.value
    ),
    global_go_counts AS (
        SELECT 
            bakta_go,
            COUNT(*) as global_count,
            COUNT(DISTINCT genome_id) as strains_with_term
        FROM {namespace}.feature_annotation
        WHERE bakta_go IS NOT NULL
        GROUP BY bakta_go
    ),
    strain_totals AS (
        SELECT 
            genome_id,
            COUNT(*) as total_annotations
        FROM {namespace}.feature_annotation
        WHERE bakta_go IS NOT NULL
        GROUP BY genome_id
    ),
    enrichment_details AS (
        SELECT 
            sgc.genome_id,
            sgc.bakta_go,
            sgc.go_name,
            sgc.count_in_strain,
            ggc.strains_with_term,
            (sgc.count_in_strain * 1.0 / st.total_annotations) / 
            (ggc.global_count * 1.0 / (SELECT SUM(total_annotations) FROM strain_totals)) as enrichment_ratio
        FROM strain_go_counts sgc
        JOIN global_go_counts ggc ON sgc.bakta_go = ggc.bakta_go
        JOIN strain_totals st ON sgc.genome_id = st.genome_id
        WHERE sgc.genome_id IN ('562.61119', '562.61097', '562.55859')  -- Top metabolic capacity strains
        AND ggc.strains_with_term >= 10
        AND sgc.count_in_strain >= 5
    )
    SELECT 
        genome_id,
        bakta_go,
        go_name,
        count_in_strain,
        strains_with_term,
        ROUND(enrichment_ratio, 2) as enrichment_ratio
    FROM enrichment_details
    WHERE enrichment_ratio > 1.5
    ORDER BY genome_id, enrichment_ratio DESC
    LIMIT 20
    """
    
    specific_df = spark.sql(specific_enrichments_query).toPandas()
    print(f"\nSpecific enriched GO terms in high-capacity strains:")
    display(specific_df)

time_query("GO Enrichment by Strain Analysis", analyze_go_enrichment_by_strain)


Executing: GO Enrichment by Strain Analysis


[Stage 466:=============================================>           (4 + 1) / 5]

Strains with the most enriched GO terms:


,genome_id,enriched_go_terms,avg_enrichment_ratio,max_enrichment_ratio
0,511145.182,4,2.7022362500,3.218846
1,562.55845,2,2.4245570000,2.629529
2,562.61197,2,2.4627295000,2.521930
3,562.61115,2,2.8530550000,3.296984
4,562.61104,1,2.4280140000,2.428014
5,562.61136,1,2.0173150000,2.017315
6,562.55864,1,2.0989730000,2.098973
7,562.61196,1,2.0825690000,2.082569
8,562.61119,1,2.0825690000,2.082569
9,562.61198,1,3.1227920000,3.122792


[Stage 500:======================>                                  (2 + 2) / 5]


Specific enriched GO terms in high-capacity strains:


,genome_id,bakta_go,go_name,count_in_strain,strains_with_term,enrichment_ratio
0,562.61097,GO:0004803,transposase activity,37,50,2.21
1,562.61119,"GO:0005198,GO:0016032,GO:0044423",None,6,39,2.08
2,562.61119,GO:0033103,protein secretion by the type VI secretion system,13,44,1.79



Query execution time: 23.05 seconds


## Summary

This notebook demonstrated:

1. **Building blocks of Query 0**: How SEED reactions, roles, term associations, and feature annotations connect to map genome features to metabolic reactions

2. **Comparative genomics insights**: Analysis of 50 E. coli strains revealed:
   - Core vs accessory genome structure
   - Functional diversity in enzymatic capabilities
   - Metabolic capacity variations
   - Pathway completeness patterns
   - Strain-specific unique features
   - Conservation patterns of molecular functions

These queries showcase the power of integrating ontology data with genomic annotations to understand bacterial diversity and evolution.